In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import tensorflow as tf
import numpy as np

import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Import our input dataset
charity_data_df = pd.read_csv('charity_data.csv')
charity_data_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(charity_data_df.dtypes)

EIN                        int64
NAME                      object
APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object


In [3]:
df = charity_data_df.drop(columns = ['EIN','NAME'])

In [4]:
# Create category_columns and numeric_columns variables
category_columns = df.dtypes[df.dtypes=='object'].index.tolist()
category_columns

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [5]:
df[category_columns].nunique()

APPLICATION_TYPE          17
AFFILIATION                6
CLASSIFICATION            71
USE_CASE                   5
ORGANIZATION               4
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
dtype: int64

In [6]:
# Binning classification
classfication_counts = df.CLASSIFICATION.value_counts()
classfication_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C1732        1
C8210        1
C1728        1
C1820        1
C2561        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
replace_classfication = list(classfication_counts[classfication_counts < 700].index)
replace_classfication

['C1700',
 'C4000',
 'C5000',
 'C1270',
 'C2700',
 'C2800',
 'C7100',
 'C1300',
 'C1280',
 'C1230',
 'C1400',
 'C7200',
 'C2300',
 'C1240',
 'C8000',
 'C7120',
 'C1500',
 'C1800',
 'C6000',
 'C1250',
 'C8200',
 'C1238',
 'C1278',
 'C1235',
 'C1237',
 'C7210',
 'C1720',
 'C2400',
 'C4100',
 'C1600',
 'C1257',
 'C0',
 'C1260',
 'C2710',
 'C1234',
 'C1246',
 'C1267',
 'C1256',
 'C3200',
 'C1283',
 'C1900',
 'C1245',
 'C1570',
 'C1236',
 'C6100',
 'C1580',
 'C1370',
 'C2150',
 'C5200',
 'C2500',
 'C4500',
 'C2570',
 'C4120',
 'C2380',
 'C2190',
 'C2170',
 'C2600',
 'C3700',
 'C1248',
 'C4200',
 'C1732',
 'C8210',
 'C1728',
 'C1820',
 'C2561']

In [8]:
for classification in replace_classfication:
    df.CLASSIFICATION = df.CLASSIFICATION.replace(classification,"Other-Classification")

df.CLASSIFICATION.value_counts()

C1000                   17326
C2000                    6074
C1200                    4837
C3000                    1918
C2100                    1883
Other-Classification     1484
C7000                     777
Name: CLASSIFICATION, dtype: int64

In [9]:
application_type_counts = df.APPLICATION_TYPE.value_counts()

In [10]:
replace_application_type = list(application_type_counts[application_type_counts < 700].index)
replace_application_type

['T10', 'T9', 'T13', 'T12', 'T2', 'T14', 'T25', 'T15', 'T29', 'T17']

In [11]:
for app_type in replace_application_type:
    df.APPLICATION_TYPE = df.APPLICATION_TYPE.replace(app_type,"Other")


df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: APPLICATION_TYPE, dtype: int64

In [12]:
income_amt_counts = df.INCOME_AMT.value_counts()
income_amt_counts

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

In [13]:
replace_income_amt = list(income_amt_counts[income_amt_counts < 300].index)
replace_income_amt

['10M-50M', '5M-10M', '50M+']

In [14]:
for income in replace_income_amt:
    df.INCOME_AMT = df.INCOME_AMT.replace(income,"5M+")

df.INCOME_AMT.value_counts()

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
5M+                564
10000-24999        543
Name: INCOME_AMT, dtype: int64

In [15]:
# Create dummy variables for the category_columns and merge on the numeric_columns to create an X dataset
df[category_columns].nunique()
df_dummies = pd.get_dummies(df[category_columns])
df_dummies

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_5M+,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
2,0,0,0,0,1,0,0,0,1,0,...,0,1,0,0,0,0,0,0,1,0
3,0,0,1,0,0,0,0,0,1,0,...,1,0,0,1,0,0,0,0,1,0
4,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
34295,0,0,0,1,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,1,0
34296,0,0,1,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,1,0
34297,0,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0


In [16]:
X = df.merge(df_dummies, left_index=True, right_index=True)
X = X.drop(category_columns, 1)
X

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_5M+,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,1,0,0,0,0,...,1,0,0,1,0,0,0,0,1,0
4,1,142590,1,0,0,1,0,0,0,0,...,1,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0


In [17]:
X = X.fillna(0)
X = X.drop(['IS_SUCCESSFUL'], 1)
X

,STATUS,ASK_AMT,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_5M+,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
1,1,108590,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
3,1,6692,0,0,1,0,0,0,0,0,...,1,0,0,1,0,0,0,0,1,0
4,1,142590,0,0,1,0,0,0,0,0,...,1,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
34296,1,5000,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
34297,1,5000,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0


In [18]:
y = df['IS_SUCCESSFUL']
y

0        1
1        1
2        0
3        1
4        1
        ..
34294    0
34295    0
34296    0
34297    1
34298    0
Name: IS_SUCCESSFUL, Length: 34299, dtype: int64

In [19]:
# Split X and y into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [20]:
# Scale X_train and X_test
# Create scaler instance
X_scaler = StandardScaler()

# Fit the scaler
X_scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
X_test_scaled

array([[ 0.01394306, -0.03099144, -0.15480007, ..., -0.12820878,
         0.02858365, -0.02858365],
       [ 0.01394306, -0.03145581, -0.15480007, ..., -0.12820878,
         0.02858365, -0.02858365],
       [ 0.01394306, -0.03145581, -0.15480007, ..., -0.12820878,
         0.02858365, -0.02858365],
       ...,
       [ 0.01394306, -0.03145581, -0.15480007, ..., -0.12820878,
         0.02858365, -0.02858365],
       [ 0.01394306, -0.03091486, -0.15480007, ..., -0.12820878,
         0.02858365, -0.02858365],
       [ 0.01394306, -0.03145581, -0.15480007, ..., -0.12820878,
         0.02858365, -0.02858365]])

In [21]:
# Create a neural network model with keras
nn = tf.keras.models.Sequential()

# Add a hidden layer with twice as many neurons as there are inputs. Use 'relu'
number_input_features = len(X.columns)
hidden_nodes_layer = 2*number_input_features

nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer, activation='relu', input_dim=number_input_features))
nn.add(tf.keras.layers.Dense(units=4, activation="relu"))

In [22]:
# add an output layer with a 'linear' activation function.
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [23]:
# print a summary of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 82)                3444      
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 332       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 5         
Total params: 3,781
Trainable params: 3,781
Non-trainable params: 0
_________________________________________________________________


In [24]:
# compile the model using the "adam" optimizer and "mean_squared_error" loss function
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [25]:
X_test_scaled_array = np.asarray(X_test_scaled)
X_train_scaled_array = np.asarray(X_train_scaled)
y_train_array = np.asarray(y_train)
y_test_array = np.asarray(y_test)

In [26]:
# train the model for 100 epochs (History)
fit_model = nn.fit(X_train_scaled_array,y_train_array,epochs=200)

Train on 25724 samples
Epoch 1/200
25724/25724 [==============================] - 2s 74us/sample - loss: 0.5721 - accuracy: 0.7186
Epoch 2/200
25724/25724 [==============================] - 1s 51us/sample - loss: 0.5547 - accuracy: 0.7289
Epoch 3/200
25724/25724 [==============================] - 1s 52us/sample - loss: 0.5524 - accuracy: 0.7285
Epoch 4/200
25724/25724 [==============================] - 1s 49us/sample - loss: 0.5510 - accuracy: 0.7294
Epoch 5/200
25724/25724 [==============================] - 1s 51us/sample - loss: 0.5493 - accuracy: 0.7297
Epoch 6/200
25724/25724 [==============================] - 1s 50us/sample - loss: 0.5480 - accuracy: 0.7305
Epoch 7/200
25724/25724 [==============================] - 1s 50us/sample - loss: 0.5479 - accuracy: 0.7300
Epoch 8/200
25724/25724 [==============================] - 1s 51us/sample - loss: 0.5470 - accuracy: 0.7312
Epoch 9/200
25724/25724 [==============================] - 1s 50us/sample - loss: 0.5466 - accuracy: 0.7324
Epoch

25724/25724 [==============================] - 1s 50us/sample - loss: 0.5384 - accuracy: 0.7372
Epoch 77/200
25724/25724 [==============================] - 1s 50us/sample - loss: 0.5380 - accuracy: 0.7374
Epoch 78/200
25724/25724 [==============================] - 1s 50us/sample - loss: 0.5379 - accuracy: 0.7381
Epoch 79/200
25724/25724 [==============================] - 1s 50us/sample - loss: 0.5378 - accuracy: 0.7378
Epoch 80/200
25724/25724 [==============================] - 1s 50us/sample - loss: 0.5380 - accuracy: 0.7367
Epoch 81/200
25724/25724 [==============================] - 1s 50us/sample - loss: 0.5375 - accuracy: 0.7371
Epoch 82/200
25724/25724 [==============================] - 1s 49us/sample - loss: 0.5381 - accuracy: 0.7378
Epoch 83/200
25724/25724 [==============================] - 1s 50us/sample - loss: 0.5377 - accuracy: 0.7374
Epoch 84/200
25724/25724 [==============================] - 1s 51us/sample - loss: 0.5375 - accuracy: 0.7375
Epoch 85/200
25724/25724 [======

In [27]:
# predict values for the train and test sets
y_train_pred = nn.predict(X_train_scaled)
y_test_pred = nn.predict(X_test_scaled)

In [28]:
# score the training predictions with r2_score()
r2_score(y_train, y_train_pred)

0.2830145164023572

In [29]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled_array,y_test_array,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/1 - 0s - loss: 0.5342 - accuracy: 0.7273
Loss: 0.5582097953193041, Accuracy: 0.7273469567298889


In [42]:
# Create a neural network model with keras
nn2 = tf.keras.models.Sequential()

# Add a hidden layer with twice as many neurons as there are inputs. Use 'relu'
number_input_features = len(X.columns)
hidden_nodes_layer1 = 2*number_input_features
hidden_nodes_layer2 = hidden_nodes_layer1

nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation='relu', input_dim=number_input_features))
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# add an output layer with a 'linear' activation function.
nn2.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# print a summary of the model
nn2.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 82)                3444      
_________________________________________________________________
dense_9 (Dense)              (None, 82)                6806      
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 83        
Total params: 10,333
Trainable params: 10,333
Non-trainable params: 0
_________________________________________________________________


In [43]:
# compile the model using the "adam" optimizer and "mean_squared_error" loss function
nn2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [44]:
# train the model for 100 epochs (History)
fit_model = nn2.fit(X_train_scaled_array,y_train_pred,epochs=100)

Train on 25724 samples
Epoch 1/100
25724/25724 [==============================] - 2s 80us/sample - loss: 0.5523 - accuracy: 3.8874e-05
Epoch 2/100
25724/25724 [==============================] - 1s 57us/sample - loss: 0.5421 - accuracy: 7.7748e-05
Epoch 3/100
25724/25724 [==============================] - 2s 60us/sample - loss: 0.5396 - accuracy: 7.7748e-05
Epoch 4/100
25724/25724 [==============================] - 1s 57us/sample - loss: 0.5376 - accuracy: 7.7748e-05
Epoch 5/100
25724/25724 [==============================] - 2s 59us/sample - loss: 0.5368 - accuracy: 7.7748e-05
Epoch 6/100
25724/25724 [==============================] - 1s 57us/sample - loss: 0.5361 - accuracy: 7.7748e-05
Epoch 7/100
25724/25724 [==============================] - 2s 59us/sample - loss: 0.5359 - accuracy: 7.7748e-05
Epoch 8/100
25724/25724 [==============================] - 1s 58us/sample - loss: 0.5354 - accuracy: 7.7748e-05
Epoch 9/100
25724/25724 [==============================] - 2s 65us/sample - loss:

25724/25724 [==============================] - 1s 55us/sample - loss: 0.5344 - accuracy: 7.7748e-05
Epoch 74/100
25724/25724 [==============================] - 1s 55us/sample - loss: 0.5344 - accuracy: 7.7748e-05
Epoch 75/100
25724/25724 [==============================] - 1s 54us/sample - loss: 0.5344 - accuracy: 7.7748e-05
Epoch 76/100
25724/25724 [==============================] - 1s 54us/sample - loss: 0.5344 - accuracy: 7.7748e-05
Epoch 77/100
25724/25724 [==============================] - 1s 54us/sample - loss: 0.5344 - accuracy: 7.7748e-05
Epoch 78/100
25724/25724 [==============================] - 1s 55us/sample - loss: 0.5344 - accuracy: 7.7748e-05
Epoch 79/100
25724/25724 [==============================] - 1s 54us/sample - loss: 0.5344 - accuracy: 7.7748e-05
Epoch 80/100
25724/25724 [==============================] - 1s 54us/sample - loss: 0.5344 - accuracy: 7.7748e-05
Epoch 81/100
25724/25724 [==============================] - 1s 55us/sample - loss: 0.5344 - accuracy: 7.7748e

In [45]:
# predict values for the train and test sets
y_train_pred = nn2.predict(X_train_scaled)
y_test_pred = nn2.predict(X_test_scaled)

# score the training predictions with r2_score()
r2_score(y_train, y_train_pred)

0.27939632341830667

In [46]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled_array,y_test_array,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/1 - 0s - loss: 0.5462 - accuracy: 0.7280
Loss: 0.5536065249887917, Accuracy: 0.7280466556549072


In [60]:
# Create a neural network model with keras
nn3 = tf.keras.models.Sequential()

# Add a hidden layer with twice as many neurons as there are inputs. Use 'relu'
number_input_features = len(X.columns)
hidden_nodes_layer1 = 2*number_input_features
hidden_nodes_layer2 = hidden_nodes_layer1
hidden_nodes_layer3 = hidden_nodes_layer2

nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation='relu', input_dim=number_input_features))
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation='relu'))

# add an output layer with a 'linear' activation function.
nn3.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# print a summary of the model
nn3.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 82)                3444      
_________________________________________________________________
dense_22 (Dense)             (None, 82)                6806      
_________________________________________________________________
dense_23 (Dense)             (None, 82)                6806      
_________________________________________________________________
dense_24 (Dense)             (None, 1)                 83        
Total params: 17,139
Trainable params: 17,139
Non-trainable params: 0
_________________________________________________________________


In [61]:
# compile the model using the "adam" optimizer and "mean_squared_error" loss function
nn3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [64]:
# train the model for 100 epochs (History)
fit_model = nn3.fit(X_train_scaled_array,y_train_pred,epochs=200)

Train on 25724 samples
Epoch 1/200
25724/25724 [==============================] - 1s 56us/sample - loss: 0.5336 - accuracy: 0.0000e+00
Epoch 2/200
25724/25724 [==============================] - 1s 55us/sample - loss: 0.5336 - accuracy: 0.0000e+00
Epoch 3/200
25724/25724 [==============================] - 2s 62us/sample - loss: 0.5336 - accuracy: 0.0000e+00
Epoch 4/200
25724/25724 [==============================] - 1s 54us/sample - loss: 0.5336 - accuracy: 0.0000e+00
Epoch 5/200
25724/25724 [==============================] - 2s 61us/sample - loss: 0.5336 - accuracy: 0.0000e+00
Epoch 6/200
25724/25724 [==============================] - 1s 56us/sample - loss: 0.5336 - accuracy: 0.0000e+00
Epoch 7/200
25724/25724 [==============================] - 2s 61us/sample - loss: 0.5336 - accuracy: 0.0000e+00
Epoch 8/200
25724/25724 [==============================] - 2s 63us/sample - loss: 0.5336 - accuracy: 0.0000e+00
Epoch 9/200
25724/25724 [==============================] - 2s 63us/sample - loss:

25724/25724 [==============================] - 2s 61us/sample - loss: 0.5336 - accuracy: 0.0000e+00
Epoch 74/200
25724/25724 [==============================] - 2s 60us/sample - loss: 0.5336 - accuracy: 0.0000e+00
Epoch 75/200
25724/25724 [==============================] - 2s 60us/sample - loss: 0.5336 - accuracy: 0.0000e+00
Epoch 76/200
25724/25724 [==============================] - 2s 60us/sample - loss: 0.5336 - accuracy: 0.0000e+00
Epoch 77/200
25724/25724 [==============================] - 2s 61us/sample - loss: 0.5336 - accuracy: 0.0000e+00
Epoch 78/200
25724/25724 [==============================] - 2s 60us/sample - loss: 0.5336 - accuracy: 0.0000e+00
Epoch 79/200
25724/25724 [==============================] - 2s 60us/sample - loss: 0.5336 - accuracy: 0.0000e+00
Epoch 80/200
25724/25724 [==============================] - 2s 60us/sample - loss: 0.5336 - accuracy: 0.0000e+00
Epoch 81/200
25724/25724 [==============================] - 2s 60us/sample - loss: 0.5336 - accuracy: 0.0000e

25724/25724 [==============================] - 2s 60us/sample - loss: 0.5336 - accuracy: 0.0000e+00
Epoch 146/200
25724/25724 [==============================] - 2s 60us/sample - loss: 0.5336 - accuracy: 0.0000e+00
Epoch 147/200
25724/25724 [==============================] - 2s 61us/sample - loss: 0.5336 - accuracy: 0.0000e+00
Epoch 148/200
25724/25724 [==============================] - 2s 60us/sample - loss: 0.5336 - accuracy: 0.0000e+00
Epoch 149/200
25724/25724 [==============================] - 2s 61us/sample - loss: 0.5336 - accuracy: 0.0000e+00
Epoch 150/200
25724/25724 [==============================] - 2s 60us/sample - loss: 0.5337 - accuracy: 0.0000e+00
Epoch 151/200
25724/25724 [==============================] - 2s 60us/sample - loss: 0.5336 - accuracy: 0.0000e+00
Epoch 152/200
25724/25724 [==============================] - 2s 60us/sample - loss: 0.5336 - accuracy: 0.0000e+00
Epoch 153/200
25724/25724 [==============================] - 2s 61us/sample - loss: 0.5336 - accuracy:

In [65]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled_array,y_test_array,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/1 - 0s - loss: 0.5460 - accuracy: 0.7280
Loss: 0.5539715990936791, Accuracy: 0.7280466556549072
